### This is the code to iterate throught subset datasets using SARIMA model to get error metrics
#### References: 
- Harris, C. R., Millman, K. J., Van Der Walt, S. J., Gommers, R., Virtanen, P., Cournapeau, D., Wieser, E., Taylor, J., Berg, S., Smith, N. J., Kern, R., Picus, M., Hoyer, S., Van Kerkwijk, M. H., Brett, M., Haldane, A., Del Río, J. F., Wiebe, M., Peterson, P., … Oliphant, T. E. (2020). Array programming with NumPy. Nature, 585(7825), 357–362. https://doi.org/10.1038/s41586-020-2649-2
- Chaudhuri, S. (2023, October 26). Assessment of Accuracy Metrics for Time Series Forecasting. Analytics Vidhya. https://medium.com/analytics-vidhya/assessment-of-accuracy-metrics-for-time-series-forecasting-bc115b655705
- Peixeiro, M. (2022). Time series forecasting in Python (Section 8.1). Manning.


#### Packages
- Package Pandas (2.2). (2024). [Python]. https://pandas.pydata.org/
- Package NumPy (1.23). (2023). [Pyhton]. https://numpy.org/
- Droettboom, J. D. H., Michael. (2024). Package matplotlib (3.8.4) [Python]. https://matplotlib.org
- Package scikit-learn (1.4). (2024). [Pyhton]. https://scikit-learn.org/stable/index.html
- Package statsmodels (0.14). (2024). [Python]. statsmodels. https://github.com/statsmodels/statsmodels


In [1]:
import pandas as pd
import numpy as np
import useful_functions as uf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from pmdarima import auto_arima

# List the files to be read - Given SARIMA does not consider features, only the original data is used
file_paths = [
    '../data/data_orig_parameters.csv'
    #'../data/BR_param_EDA.csv',
    #'../data/data_cleaned_RF.csv',
    #'../data/data_cleaned_LASSO.csv',
    #'../data/data_cleaned_RFE.csv'
]

# List of outlier thresholds to be tested
outlier_thresholds = [np.nan, 0.05, 0.10, 0.15, 0.20]

# Create a dictionary to store the errors
errors_dict = {}

# Loop through the files and outlier thresholds
for file_path in file_paths:
    print(f"REading File: {file_path}") # Print the file being read
    for remove_outliers_threshold in outlier_thresholds: # For each outlier threshold
        print(f"Outlier Threshold: {remove_outliers_threshold}")
        # Load  data
        df_raw = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
        target_variable = df_raw.columns[0]
        df = df_raw[[target_variable]] # Only the target variable is used

        # Remove outliers using the threshold
        if not pd.isna(remove_outliers_threshold):
            df_cleaned = uf.remove_outliers(df.copy(), threshold=remove_outliers_threshold)
        else: # If the threshold is NaN, the data is not cleaned
            df_cleaned = df.copy()

        # The outliers removal may have created missing values, which are filled with the mean
        df_adjusted = uf.fill_missing_values(df_cleaned)

        # Split the data into train and test sets
        test_size = 48  # meses
        df_train = df_adjusted[:-test_size]
        df_test = df_adjusted[-test_size:]

        # Model training
        auto_model = auto_arima(df_train[target_variable], # For each threshold, the best model is selected
                                start_p=0, start_q=0, 
                                max_p=12, max_q=12, 
                                m=12, 
                                start_P=0, start_Q=0, 
                                max_P=12, max_Q=12, 
                                seasonal=True, 
                                d=1, D=0,  
                                trace=False,
                                error_action='ignore',  
                                suppress_warnings=True, 
                                stepwise=True, max_order=10)
        model = SARIMAX(df_train, order=auto_model.order, seasonal_order=auto_model.seasonal_order)
        
        # In case you need to test an specific model, uncomment the line below and comment the line above
        #model = SARIMAX(df_train, order=(4,1,4), seasonal_order=(1,0,0,12))
        
        model_fit = model.fit(disp=False, maxiter=200)
        predictions = model_fit.forecast(steps=len(df_test)) # Predict on the test set

        # Calculate the errors
        mape = mean_absolute_percentage_error(df_test, predictions)
        rmse = np.sqrt(mean_squared_error(df_test, predictions))
        mae = mean_absolute_error(df_test, predictions)

        # Save the errors in the dictionary and the model summary
        errors_dict[(file_path, remove_outliers_threshold)] = {'MAPE': mape, 'RMSE': rmse, 'MAE': mae, "Model_summary": model_fit.summary()}

# Print the errors
for key, value in errors_dict.items():
    print(f"File: {key[0]}, Outlier Threshold: {key[1]} -> Errors: {value}")


REading File: ../data/data_orig_parameters.csv
Outlier Threshold: nan


g:\My Drive\Data_Projects\MDS\master_thesis\fiscal-balance-forecast\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
g:\My Drive\Data_Projects\MDS\master_thesis\fiscal-balance-forecast\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Outlier Threshold: 0.05


g:\My Drive\Data_Projects\MDS\master_thesis\fiscal-balance-forecast\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
g:\My Drive\Data_Projects\MDS\master_thesis\fiscal-balance-forecast\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Outlier Threshold: 0.1


...

Outlier Threshold: 0.15


g:\My Drive\Data_Projects\MDS\master_thesis\fiscal-balance-forecast\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
g:\My Drive\Data_Projects\MDS\master_thesis\fiscal-balance-forecast\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Outlier Threshold: 0.2


g:\My Drive\Data_Projects\MDS\master_thesis\fiscal-balance-forecast\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
g:\My Drive\Data_Projects\MDS\master_thesis\fiscal-balance-forecast\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
g:\My Drive\Data_Projects\MDS\master_thesis\fiscal-balance-forecast\.venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
g:\My Drive\Data_Projects\MDS\master_thesis\fiscal-balance-forecast\.venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters foun

File: ../data/data_orig_parameters.csv, Outlier Threshold: nan -> Errors: {'MAPE': 1.6386543380354326, 'RMSE': 49195.99195074552, 'MAE': 34046.33020784072, 'Model_summary': <class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
Dep. Variable:             ECO_fiscal_result_month   No. Observations:                  228
Model:             SARIMAX(4, 1, 4)x(1, 0, [], 12)   Log Likelihood               -2393.699
Date:                             Mon, 15 Apr 2024   AIC                           4807.398
Time:                                     13:43:05   BIC                           4841.647
Sample:                                 01-01-2001   HQIC                          4821.218
                                      - 12-01-2019                                         
Covariance Type:                               opg                                         
                 coef    std err          z

In [2]:
#Export the output to a txt file to check the results and model summary
with open('output_SARIMA_subsets.txt', 'w') as f:
    for key, value in errors_dict.items():
        f.write(f"File: {key[0]}, Outlier Threshold: {key[1]} -> Errors: {value}")
        f.write('\n')
        f.write('\n')